In [ ]:
%load_ext autoreload
%autoreload 2

import jax.numpy as jnp
import jax.random as jrandom
import optax
import numpy as np
import matplotlib.pyplot as plt

from sampling_utils import get_samples, get_z_samples
from reparameterization import get_rp_gradients, objective
from likelihoodratio import get_lr_gradients
from constants import RAND_KEY
from training import fit

#### Initial Parameters

In [ ]:
mu_0 = 1.
V_0 = 0.0001

A = 1.5
B = 1.

C = 1.
E = .000001

In [ ]:
zs, xs = get_samples(100, 1000, mu_0, V_0, A, B, C, E)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(zs.mean(axis=1))
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.scatter(xs[:-1], xs[1:], color="purple", label="x")
plt.scatter(zs[:-1], zs[1:], color="orange", label="z")
plt.legend()
plt.show()

## Find optimal A using gradient estimators

We try to maximise the marginal likelihood $\max_{A} \mathbb{E}_{p(Z|A, B, \mu_0, V_0)} [p(X|Z, C, E)]$ w.r.t $A$, by using LR and RP gradient estimators together with an Adam optimiser until convergence.

We assume here all other parameters ($\mu_0, V_0, B, C, E$) are known.

In [ ]:
N = 1000000
num_inputs = 2
NUM_TRAIN_STEPS = 3000
A = 1.5

# Set to False for RP gradient estimator
LR_ESTIMATOR = True 

A_n = 1.
_, xs = get_samples(num_inputs, N, mu_0, V_0, A, B, C, E)

init_param = jnp.array([A_n])


optimizer = optax.chain(
    optax.adam(learning_rate=0.0005),
    optax.scale(-1.0)
)

optimizer.init(init_param)

learned_params, losses, gradients = fit(
    params=init_param,
    optimizer=optimizer, 
    training_steps=NUM_TRAIN_STEPS, 
    mu0=mu_0, V0=V_0, B=B, C=C, E=E, xs=xs, 
    num_inputs=num_inputs, 
    N=N, 
    lr_estimator=LR_ESTIMATOR,
)

In [ ]:
N = 1000000
As = np.linspace(-4, 4, 100)

_, xs = get_samples(num_inputs, N, mu_0, V_0, A, B, C, E)
lr_grads = np.zeros(100,)
rp_grads = np.zeros(100,)
obj = np.zeros(100,)
key = RAND_KEY
for i in range(100):
    a = As[i]
    key, subkey = jrandom.split(key)
    zs, epsilons = get_z_samples(num_inputs, N, mu_0, V_0, a, B, subkey)
    lr_grads[i] = get_lr_gradients(mu_0, V_0, a, B, C, E, xs, zs)
    rp_grads[i] = get_rp_gradients(mu_0, V_0, a, B, C, E, epsilons, xs)
    obj[i] = objective(mu_0, V_0, a, B, C, E, epsilons, xs)

print(f"Max marginal likelihood reached by RP gradients: {obj[np.argmin(list(map(lambda x: abs(x - 0), rp_grads)))]:.4f}")
print(f"Max marginal likelihood reached by LR gradients: {obj[np.argmin(list(map(lambda x: abs(x - 0), lr_grads)))]:.4f}")
print(f"Marginal likelihood of true A: {max(obj):.4f}")

plt.figure(figsize=(6,6))
plt.plot(As, obj, color="green", label="Marginal likelihood")
plt.plot(As, lr_grads, color="purple", label="LR grads")
plt.plot(As, rp_grads, color="cornflowerblue", label="RP gradients")
plt.legend()
plt.show()

#### LDS produced with learned parameter

In [ ]:
learned_A = learned_params[0]
zs, xs = get_samples(num_inputs, N, mu_0, V_0, learned_A, B, C, E)

plt.figure(figsize=(15,5))
plt.scatter(xs[:-1], xs[1:], color="purple", label="x")
plt.scatter(zs[:-1], zs[1:], color="orange", label="z")
plt.legend()
plt.show()